# Setup

In [1]:
import sys
if '/home/hungnp5/ecommerce-datawarehouse' not in sys.path:
    sys.path.append('/home/hungnp5/ecommerce-datawarehouse')
from helpers.config.spark_init import init_spark
from helpers.util.utils import fetch_sql_table, write_sql_table

In [2]:
import pyspark.sql.functions as f
from pyspark.sql.functions import col, count
from pyspark.sql import Window
from functools import reduce

In [3]:
spark, sqlContext = init_spark(application_name='test', cores_total=40, cors_per_executors=10)

# Fetch Data

In [4]:
oltp_customers = fetch_sql_table(spark=spark, db='OLTP', db_table='OLTP_CUSTOMERS')
oltp_categories_translation = fetch_sql_table(spark=spark, db='OLTP', db_table='OLTP_CATEGORIES_TRANSLATION')
oltp_geolocation = fetch_sql_table(spark=spark, db='OLTP', db_table='OLTP_GEOLOCATION')
oltp_order_items = fetch_sql_table(spark=spark, db='OLTP', db_table='OLTP_ORDER_ITEMS')
oltp_order_payments = fetch_sql_table(spark=spark, db='OLTP', db_table='OLTP_ORDER_PAYMENTS')
oltp_order_reviews = fetch_sql_table(spark=spark, db='OLTP', db_table='OLTP_ORDER_REVIEWS')
oltp_orders = fetch_sql_table(spark=spark, db='OLTP', db_table='OLTP_ORDERS')
oltp_products = fetch_sql_table(spark=spark, db='OLTP', db_table='OLTP_PRODUCTS')
oltp_sellers = fetch_sql_table(spark=spark, db='OLTP', db_table='OLTP_SELLERS')



# Functions

In [5]:
def check_null(df):
    df.select([count(f.when(col(c).isNull(), c)).alias(c) for c in df.columns]).show()

In [6]:
def blank_as_null(x):
    return f.when(col(x) != "", col(x)).otherwise(None)
def replace_empty_string_with_null(dataframe):
    return reduce(lambda df, x: df.withColumn(x, blank_as_null(x)), [name for name, type in dataframe.dtypes if type == 'string'], dataframe)

In [7]:
def check_duplicates(df, cols):
    df.groupby(cols).count().where('count > 1').show(truncate=False)

# Preprocess

## Geolocation
1. Some outliers coordinates that are outside brazillian territory

In [8]:
oltp_geolocation.describe().show()

+-------+---------------------------+-------------------+-------------------+----------------+-----------------+
|summary|GEOLOCATION_ZIP_CODE_PREFIX|    GEOLOCATION_LAT|    GEOLOCATION_LNG|GEOLOCATION_CITY|GEOLOCATION_STATE|
+-------+---------------------------+-------------------+-------------------+----------------+-----------------+
|  count|                    1000163|            1000163|            1000163|         1000163|          1000163|
|   mean|          36574.16646586607|-21.176152910383102|-46.390541320935995|            null|             null|
| stddev|          30549.33571031949|  5.715866308823084|  4.269748306619793|            null|             null|
|    min|                      01001|  -36.6053744107061|-101.46676644931476|        * cidade|               AC|
|    max|                      99990|  45.06593318269697| 121.10539381057764|            óleo|               TO|
+-------+---------------------------+-------------------+-------------------+----------------+--

In [9]:
#Brazils most Northern spot is at 5 deg 16′ 27.8″ N latitude.;
#it’s most Western spot is at 73 deg, 58′ 58.19″W Long.
#It’s most southern spot is at 33 deg, 45′ 04.21″ S Latitude.
#It’s most Eastern spot is 34 deg, 47′ 35.33″ W Long.
oltp_geolocation_remove_outliers = oltp_geolocation.where('GEOLOCATION_LAT <= 5.27438888')\
    .where('GEOLOCATION_LNG >= -73.98283055')\
    .where('GEOLOCATION_LAT >=  -33.75116944')\
    .where('GEOLOCATION_LNG <= -34.79314722')
oltp_geolocation_remove_outliers.show()

+---------------------------+-------------------+-------------------+----------------+-----------------+
|GEOLOCATION_ZIP_CODE_PREFIX|    GEOLOCATION_LAT|    GEOLOCATION_LNG|GEOLOCATION_CITY|GEOLOCATION_STATE|
+---------------------------+-------------------+-------------------+----------------+-----------------+
|                      01037| -23.54562128115268| -46.63929204800168|       sao paulo|               SP|
|                      01046|-23.546081127035535| -46.64482029837157|       sao paulo|               SP|
|                      01046| -23.54612896641469| -46.64295148361138|       sao paulo|               SP|
|                      01041|  -23.5443921648681| -46.63949930627844|       sao paulo|               SP|
|                      01035|-23.541577961711493| -46.64160722329613|       sao paulo|               SP|
|                      01012|-23.547762303364266| -46.63536053788448|       são paulo|               SP|
|                      01047|-23.546273112412678| -46.6

## Product
1. Replace empty string with null
2. Check null
3. Check duplicate

In [10]:
oltp_products_processed = replace_empty_string_with_null(oltp_products)
oltp_categories_translation_processed = replace_empty_string_with_null(oltp_categories_translation)

In [11]:
check_null(oltp_products_processed)
check_null(oltp_categories_translation_processed)

+----------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+
|PRODUCT_ID|PRODUCT_CATEGORY_NAME|PRODUCT_NAME_LENGTH|PRODUCT_DESCRIPTION_LENGTH|PRODUCT_PHOTOS_QTY|PRODUCT_WEIGHT_G|PRODUCT_LENGTH_CM|PRODUCT_HEIGHT_CM|PRODUCT_WIDTH_CM|
+----------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+
|         0|                  610|                  0|                         0|                 0|               0|                0|                0|               0|
+----------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+

+---------------------+-----------------------------+
|PRODUCT_CATEGORY_NAME|PRODUCT_CATEGORY_NAME_ENGLISH|
+---------------------+-------------

In [12]:
# Duplicates
check_duplicates(oltp_products_processed, cols=['PRODUCT_ID'])
check_duplicates(oltp_categories_translation_processed, cols=['PRODUCT_CATEGORY_NAME'])
check_duplicates(oltp_categories_translation_processed, cols=['PRODUCT_CATEGORY_NAME_ENGLISH'])

+----------+-----+
|PRODUCT_ID|count|
+----------+-----+
+----------+-----+

+---------------------+-----+
|PRODUCT_CATEGORY_NAME|count|
+---------------------+-----+
+---------------------+-----+

+-----------------------------+-----+
|PRODUCT_CATEGORY_NAME_ENGLISH|count|
+-----------------------------+-----+
+-----------------------------+-----+



## Seller
1. Replace emptry string with null
2. Check null
3. Check dup

In [13]:
oltp_sellers_processed = replace_empty_string_with_null(oltp_sellers)
check_null(oltp_sellers_processed)
check_duplicates(oltp_sellers_processed, cols=['SELLER_ID'])

+---------+----------------------+-----------+------------+
|SELLER_ID|SELLER_ZIP_CODE_PREFIX|SELLER_CITY|SELLER_STATE|
+---------+----------------------+-----------+------------+
|        0|                     0|          0|           0|
+---------+----------------------+-----------+------------+

+---------+-----+
|SELLER_ID|count|
+---------+-----+
+---------+-----+



## Payment
1. Replace emptry string with null
2. Check null
3. Check dup*

* Payment Sequential: a customer may pay an order with more than one payment method. If he does so, a sequence will be created to accommodate all payments.

In [14]:
oltp_order_payments_processed = replace_empty_string_with_null(oltp_order_payments)
check_null(oltp_order_payments_processed)
check_duplicates(oltp_order_payments_processed, cols=['ORDER_ID', 'PAYMENT_SEQUENTIAL'])

+--------+------------------+------------+--------------------+-------------+
|ORDER_ID|PAYMENT_SEQUENTIAL|PAYMENT_TYPE|PAYMENT_INSTALLMENTS|PAYMENT_VALUE|
+--------+------------------+------------+--------------------+-------------+
|       0|                 0|           0|                   0|            0|
+--------+------------------+------------+--------------------+-------------+

+--------+------------------+-----+
|ORDER_ID|PAYMENT_SEQUENTIAL|count|
+--------+------------------+-----+
+--------+------------------+-----+



In [15]:
oltp_order_payments_processed.where('ORDER_ID == "8ca5bdac5ebe8f2d6fc9171d5ebc906a"').show()

+--------------------+------------------+------------+--------------------+-------------+
|            ORDER_ID|PAYMENT_SEQUENTIAL|PAYMENT_TYPE|PAYMENT_INSTALLMENTS|PAYMENT_VALUE|
+--------------------+------------------+------------+--------------------+-------------+
|8ca5bdac5ebe8f2d6...|                 3|     voucher|                   1|         15.0|
|8ca5bdac5ebe8f2d6...|                 7|     voucher|                   1|         15.0|
|8ca5bdac5ebe8f2d6...|                 5|     voucher|                   1|         15.0|
|8ca5bdac5ebe8f2d6...|                 2|     voucher|                   1|         15.0|
|8ca5bdac5ebe8f2d6...|                 1| credit_card|                   2|        59.08|
|8ca5bdac5ebe8f2d6...|                 6|     voucher|                   1|         15.0|
|8ca5bdac5ebe8f2d6...|                 8|     voucher|                   1|         15.0|
|8ca5bdac5ebe8f2d6...|                 4|     voucher|                   1|         15.0|
|8ca5bdac5

# ETL

## TO STAGING_CUSTOMERS
1. Left-Join customers and geolocation by zipcode-prefix
2. One zipcode contains many lat, lng -> we need to choose one by using row_numbering to avoid duplicate user 
3. Check-null
4. Check-duplicate
5. Save

In [35]:
oltp_customers.show(2)
oltp_geolocation_remove_outliers.show(2)

+--------------------+--------------------+------------------------+--------------------+--------------+
|         CUSTOMER_ID|  CUSTOMER_UNIQUE_ID|CUSTOMER_ZIP_CODE_PREFIX|       CUSTOMER_CITY|CUSTOMER_STATE|
+--------------------+--------------------+------------------------+--------------------+--------------+
|06b8999e2fba1a1fb...|861eff4711a542e4b...|                   14409|              franca|            SP|
|18955e83d337fd6b2...|290c77bc529b7ac93...|                   09790|sao bernardo do c...|            SP|
+--------------------+--------------------+------------------------+--------------------+--------------+
only showing top 2 rows

+---------------------------+-------------------+------------------+----------------+-----------------+
|GEOLOCATION_ZIP_CODE_PREFIX|    GEOLOCATION_LAT|   GEOLOCATION_LNG|GEOLOCATION_CITY|GEOLOCATION_STATE|
+---------------------------+-------------------+------------------+----------------+-----------------+
|                      01037| -23

In [36]:
staging_customers = oltp_customers.alias('a').join(oltp_geolocation_remove_outliers.alias('b'), col('a.CUSTOMER_ZIP_CODE_PREFIX') == col('b.GEOLOCATION_ZIP_CODE_PREFIX'), how='left')\
    .withColumn('rank', f.row_number().over(Window.partitionBy('CUSTOMER_UNIQUE_ID').orderBy('GEOLOCATION_LAT', 'GEOLOCATION_LNG')))\
    .where('rank == 1')\
    .select('CUSTOMER_UNIQUE_ID', 'CUSTOMER_ZIP_CODE_PREFIX', 'CUSTOMER_CITY', 'CUSTOMER_STATE', 'GEOLOCATION_LAT', 'GEOLOCATION_LNG')\
    .withColumnRenamed('CUSTOMER_ZIP_CODE_PREFIX', 'CUSTOMER_ZIPCODE')\
    .withColumnRenamed('GEOLOCATION_LAT', 'CUSTOMER_GEO_LAT')\
    .withColumnRenamed('GEOLOCATION_LNG', 'CUSTOMER_GEO_LNG')
staging_customers.show(2, truncate=False)

+--------------------------------+----------------+--------------+--------------+-------------------+-------------------+
|CUSTOMER_UNIQUE_ID              |CUSTOMER_ZIPCODE|CUSTOMER_CITY |CUSTOMER_STATE|CUSTOMER_GEO_LAT   |CUSTOMER_GEO_LNG   |
+--------------------------------+----------------+--------------+--------------+-------------------+-------------------+
|000c8bdb58a29e7115cfc257230fb21b|31555           |belo horizonte|MG            |-19.849090315309482|-43.98182678974558 |
|0078bb0f0d23e922d08437b7d0e13907|31720           |belo horizonte|MG            |-19.841509703955   |-43.949346506205146|
+--------------------------------+----------------+--------------+--------------+-------------------+-------------------+
only showing top 2 rows



In [37]:
check_null(staging_customers)
check_duplicates(staging_customers, cols=['CUSTOMER_UNIQUE_ID'])

+------------------+----------------+-------------+--------------+----------------+----------------+
|CUSTOMER_UNIQUE_ID|CUSTOMER_ZIPCODE|CUSTOMER_CITY|CUSTOMER_STATE|CUSTOMER_GEO_LAT|CUSTOMER_GEO_LNG|
+------------------+----------------+-------------+--------------+----------------+----------------+
|                 0|               0|            0|             0|             270|             270|
+------------------+----------------+-------------+--------------+----------------+----------------+

+------------------+-----+
|CUSTOMER_UNIQUE_ID|count|
+------------------+-----+
+------------------+-----+



In [ ]:
write_sql_table(staging_customers, db='STAGING', db_table='STAGING_CUSTOMERS')

## To Staging Products
1. Left-Join Products and Product Name Translation
2. Check-null
3. Check-dup
4. Save

In [22]:
oltp_products_processed.show(2)
oltp_categories_translation_processed.show(2)

+--------------------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+
|          PRODUCT_ID|PRODUCT_CATEGORY_NAME|PRODUCT_NAME_LENGTH|PRODUCT_DESCRIPTION_LENGTH|PRODUCT_PHOTOS_QTY|PRODUCT_WEIGHT_G|PRODUCT_LENGTH_CM|PRODUCT_HEIGHT_CM|PRODUCT_WIDTH_CM|
+--------------------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+
|1e9e8ef04dbcff454...|           perfumaria|                 40|                       287|                 1|             225|               16|               10|              14|
|3aa071139cb16b67c...|                artes|                 44|                       276|                 1|            1000|               30|               18|              20|
+--------------------+---------------------+-------------------+--------------------------+----

In [27]:
staging_product = oltp_products_processed.join(oltp_categories_translation_processed, on='PRODUCT_CATEGORY_NAME', how='left')\
    .withColumnRenamed('PRODUCT_WEIGHT_G', 'PRODUCT_WEIGHT_GRAMS')\
    .select('PRODUCT_ID', 'PRODUCT_CATEGORY_NAME', 'PRODUCT_CATEGORY_NAME_ENGLISH', 'PRODUCT_NAME_LENGTH', 'PRODUCT_DESCRIPTION_LENGTH', 'PRODUCT_PHOTOS_QTY', 'PRODUCT_WEIGHT_GRAMS', 'PRODUCT_LENGTH_CM', 'PRODUCT_HEIGHT_CM','PRODUCT_WIDTH_CM')
staging_product.show(2)

+--------------------+---------------------+-----------------------------+-------------------+--------------------------+------------------+--------------------+-----------------+-----------------+----------------+
|          PRODUCT_ID|PRODUCT_CATEGORY_NAME|PRODUCT_CATEGORY_NAME_ENGLISH|PRODUCT_NAME_LENGTH|PRODUCT_DESCRIPTION_LENGTH|PRODUCT_PHOTOS_QTY|PRODUCT_WEIGHT_GRAMS|PRODUCT_LENGTH_CM|PRODUCT_HEIGHT_CM|PRODUCT_WIDTH_CM|
+--------------------+---------------------+-----------------------------+-------------------+--------------------------+------------------+--------------------+-----------------+-----------------+----------------+
|1e5428c428e0f783a...|                  pcs|                    computers|                 55|                      1067|                 1|               12000|               27|               50|              53|
|9aedf557945e816d4...|                  pcs|                    computers|                 55|                      3610|                 4|

In [30]:
check_null(staging_product)
check_duplicates(staging_product, cols=['PRODUCT_ID'])

+----------+---------------------+-----------------------------+-------------------+--------------------------+------------------+--------------------+-----------------+-----------------+----------------+
|PRODUCT_ID|PRODUCT_CATEGORY_NAME|PRODUCT_CATEGORY_NAME_ENGLISH|PRODUCT_NAME_LENGTH|PRODUCT_DESCRIPTION_LENGTH|PRODUCT_PHOTOS_QTY|PRODUCT_WEIGHT_GRAMS|PRODUCT_LENGTH_CM|PRODUCT_HEIGHT_CM|PRODUCT_WIDTH_CM|
+----------+---------------------+-----------------------------+-------------------+--------------------------+------------------+--------------------+-----------------+-----------------+----------------+
|         0|                  610|                          623|                  0|                         0|                 0|                   0|                0|                0|               0|
+----------+---------------------+-----------------------------+-------------------+--------------------------+------------------+--------------------+-----------------+-----------

In [31]:
write_sql_table(staging_product, db='STAGING', db_table='STAGING_PRODUCTS')

## To Staging Sellers
1. Left-Join Sellers with Geolocation
2. Row number zip-code -> avoid user duplication
3. Check null
4. Check duplication
5. Save

In [38]:
oltp_sellers.show(2)
oltp_geolocation_remove_outliers.show(2)

+--------------------+----------------------+-----------+------------+
|           SELLER_ID|SELLER_ZIP_CODE_PREFIX|SELLER_CITY|SELLER_STATE|
+--------------------+----------------------+-----------+------------+
|3442f8959a84dea7e...|                 13023|   campinas|          SP|
|d1b65fc7debc3361e...|                 13844| mogi guacu|          SP|
+--------------------+----------------------+-----------+------------+
only showing top 2 rows

+---------------------------+-------------------+------------------+----------------+-----------------+
|GEOLOCATION_ZIP_CODE_PREFIX|    GEOLOCATION_LAT|   GEOLOCATION_LNG|GEOLOCATION_CITY|GEOLOCATION_STATE|
+---------------------------+-------------------+------------------+----------------+-----------------+
|                      01037| -23.54562128115268|-46.63929204800168|       sao paulo|               SP|
|                      01046|-23.546081127035535|-46.64482029837157|       sao paulo|               SP|
+---------------------------+

In [46]:
staging_sellers = oltp_sellers_processed.alias('a').join(oltp_geolocation_remove_outliers.alias('b'), col('a.SELLER_ZIP_CODE_PREFIX') == col('b.GEOLOCATION_ZIP_CODE_PREFIX'), how='left')\
    .withColumn('rank', f.row_number().over(Window.partitionBy('SELLER_ID').orderBy('GEOLOCATION_LAT', 'GEOLOCATION_LNG')))\
    .where('rank == 1')\
    .select('SELLER_ID', 'SELLER_ZIP_CODE_PREFIX', 'SELLER_CITY', 'SELLER_STATE', 'GEOLOCATION_LAT', 'GEOLOCATION_LNG')\
    .withColumnRenamed('SELLER_ZIP_CODE_PREFIX', 'SELLER_ZIPCODE')\
    .withColumnRenamed('GEOLOCATION_LAT', 'SELLER_GEO_LAT')\
    .withColumnRenamed('GEOLOCATION_LNG', 'SELLER_GEO_LONG')
staging_sellers.show(2, truncate=False)

+--------------------------------+--------------+-----------+------------+-------------------+-------------------+
|SELLER_ID                       |SELLER_ZIPCODE|SELLER_CITY|SELLER_STATE|SELLER_GEO_LAT     |SELLER_GEO_LONG    |
+--------------------------------+--------------+-----------+------------+-------------------+-------------------+
|062ce95fa2ad4dfaedfc79260130565f|95913         |lajeado    |RS          |-29.455996441348013|-51.948006819394465|
|0b64bcdb0784abc139af04077d49a20e|92420         |canoas     |RS          |-29.89971549769505 |-51.23310201482481 |
+--------------------------------+--------------+-----------+------------+-------------------+-------------------+
only showing top 2 rows



In [48]:
check_null(staging_sellers)
check_duplicates(staging_sellers, cols=['SELLER_ID'])

+---------+--------------+-----------+------------+--------------+---------------+
|SELLER_ID|SELLER_ZIPCODE|SELLER_CITY|SELLER_STATE|SELLER_GEO_LAT|SELLER_GEO_LONG|
+---------+--------------+-----------+------------+--------------+---------------+
|        0|             0|          0|           0|             7|              7|
+---------+--------------+-----------+------------+--------------+---------------+

+---------+-----+
|SELLER_ID|count|
+---------+-----+
+---------+-----+



In [49]:
write_sql_table(staging_sellers, db='STAGING', db_table='STAGING_SELLERS')

## To Staging Payments
1. Save

In [56]:
write_sql_table(oltp_order_payments_processed, db='STAGING', db_table='STAGING_ORDER_PAYMENTS')

# To Staging Orders
1. Left join
2. Check null
3. Check dup
4. Save/

In [16]:
oltp_orders.show(2)
oltp_order_items.show(2)
oltp_order_reviews.show(2)
oltp_customers.show(2)

+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|            ORDER_ID|         CUSTOMER_ID|ORDER_STATUS|ORDER_PURCHASE_TIMESTAMP|  ORDER_APPROVAL_AT|ORDER_DELIVERED_CARRIER_DATE|ORDER_DELIVERED_CUSTOMER_DATE|ORDER_ESTIMATED_DELIVERY_DATE|
+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|e481f51cbdc54678b...|9ef432eb625129730...|   delivered|     2017-10-02 10:56:33|2017-10-02 11:07:15|         2017-10-04 19:55:00|          2017-10-10 21:25:13|          2017-10-18 00:00:00|
|53cdb2fc8bc7dce0b...|b0830fb4747a6c6d2...|   delivered|     2018-07-24 20:41:37|2018-07-26 03:24:27|         2018-07-26 14:31:00|          2018-08-07 15:27:45|          2018-08-13 00:00:00|
+--------------------+--------------------+--

In [19]:
staging_orders = oltp_orders.join(oltp_order_items, on='ORDER_ID', how='left')\
    .join(oltp_order_reviews, on='ORDER_ID', how='left')\
    .join(oltp_customers, on='CUSTOMER_ID', how='left')\
    .select('ORDER_ID', "CUSTOMER_UNIQUE_ID", "PRODUCT_ID", "SELLER_ID", "ORDER_ITEM_ID", "PRICE", "FREIGHT_VALUE", \
        "ORDER_STATUS", "REVIEW_SCORE", "ORDER_PURCHASE_TIMESTAMP", "ORDER_APPROVAL_AT", "ORDER_DELIVERED_CARRIER_DATE", "ORDER_DELIVERED_CUSTOMER_DATE", "ORDER_ESTIMATED_DELIVERY_DATE")\
    .withColumnRenamed('ORDER_ITEM_ID', 'ORDER_ITEM_QTY')\
    .withColumnRenamed('ORDER_APPROVAL_AT', 'ORDER_APPROVAL_TIMESTAMP')\
    .withColumnRenamed('ORDER_DELIVERED_CARRIER_DATE', 'ORDER_DELIVERED_CARRIER_TIMESTAMP')\
    .withColumnRenamed('ORDER_DELIVERED_CUSTOMER_DATE', 'ORDER_DELIVERED_CUSTOMER_TIMESTAMP')\
    .withColumnRenamed('ORDER_ESTIMATED_DELIVERY_DATE', 'ORDER_ESTIMATED_DELIVERY_TIMESTAMP')\
    .cache()

In [22]:
staging_orders.show(1)

+--------------------+--------------------+--------------------+--------------------+--------------+-----+-------------+------------+------------+------------------------+------------------------+---------------------------------+----------------------------------+----------------------------------+
|            ORDER_ID|  CUSTOMER_UNIQUE_ID|          PRODUCT_ID|           SELLER_ID|ORDER_ITEM_QTY|PRICE|FREIGHT_VALUE|ORDER_STATUS|REVIEW_SCORE|ORDER_PURCHASE_TIMESTAMP|ORDER_APPROVAL_TIMESTAMP|ORDER_DELIVERED_CARRIER_TIMESTAMP|ORDER_DELIVERED_CUSTOMER_TIMESTAMP|ORDER_ESTIMATED_DELIVERY_TIMESTAMP|
+--------------------+--------------------+--------------------+--------------------+--------------+-----+-------------+------------+------------+------------------------+------------------------+---------------------------------+----------------------------------+----------------------------------+
|975a0290ce9c93595...|2e5dcf79b225e8d16...|d435c3fa55cfd37fa...|51ed72984a04e5dd3...|            

In [23]:
staging_orders_processed = replace_empty_string_with_null(staging_orders)

In [24]:
check_null(staging_orders_processed)

+--------+------------------+----------+---------+--------------+-----+-------------+------------+------------+------------------------+------------------------+---------------------------------+----------------------------------+----------------------------------+
|ORDER_ID|CUSTOMER_UNIQUE_ID|PRODUCT_ID|SELLER_ID|ORDER_ITEM_QTY|PRICE|FREIGHT_VALUE|ORDER_STATUS|REVIEW_SCORE|ORDER_PURCHASE_TIMESTAMP|ORDER_APPROVAL_TIMESTAMP|ORDER_DELIVERED_CARRIER_TIMESTAMP|ORDER_DELIVERED_CUSTOMER_TIMESTAMP|ORDER_ESTIMATED_DELIVERY_TIMESTAMP|
+--------+------------------+----------+---------+--------------+-----+-------------+------------+------------+------------------------+------------------------+---------------------------------+----------------------------------+----------------------------------+
|       0|                 0|       778|      778|           778|  778|          778|           0|           1|                       0|                     162|                             1980|       

In [30]:
# Check if 1 order-id have > 1 customers
staging_orders_processed.groupby('ORDER_ID').agg(f.countDistinct('CUSTOMER_UNIQUE_ID').alias('count_distinct')).where('count_distinct > 1').show()

+--------+--------------+
|ORDER_ID|count_distinct|
+--------+--------------+
+--------+--------------+



In [31]:
write_sql_table(staging_orders_processed, db='STAGING', db_table='STAGING_ORDERS')